In [1]:
import pandas as pd
import zipfile

In [2]:
# define file names
zip_file_path = "parsed_data.zip"
csv_file_name = "parsed_data/{}parsed_data.csv"
years = range(2019,2024)

In [3]:
def read_csv_from_zip(zip_file_path, csv_file_name):
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        with z.open(csv_file_name) as f:
            return pd.read_csv(f)

In [4]:
single_year_dfs = []
# process each year separately to concatenate later
for year in years:
    print (f"\nProcessing {year}")
    df = read_csv_from_zip(zip_file_path, csv_file_name.format(year))
    print (f"Data frame has {len(df)} rows")
    # add standard columns
    df["PLT_ID"] = df["plt_id"]
    df["ROW"] = df["row"]
    df["COL"] =df["column"]
    df["TEST_LOC"] = "Dallas, TX"
    df["ENTRY_NAME"] = df["entity_name"]
    df["ENTRY_CODE"] = df["entry_code"]
    df["TRAIT"] = "Turf Quality"
    # find 'quality' columns and transpose df
    quality_cols = [col_name for col_name in df.columns if col_name.startswith("quality_")]
    print (f"Quality columns {quality_cols}")
    keep_columns = ["PLT_ID", "ROW", "COL",  "TEST_LOC", "ENTRY_NAME", "ENTRY_CODE", "TRAIT"]
    melted = pd.melt(
        df, id_vars=keep_columns, value_vars=quality_cols, var_name='DATE', value_name='QUALITY'
    )
    melted["QUALITY"] = pd.to_numeric(melted["QUALITY"], errors="coerce")
    # Convert to datetime and format as mm/dd/yy
    melted["DATE"] = melted["DATE"].str.extract(r"(\d{8})")
    melted["DATE"] = pd.to_datetime(melted["DATE"], format="%Y%m%d").dt.strftime("%m/%d/%y")
    melted["YEAR"] = pd.to_datetime(melted["DATE"], format="%m/%d/%y").dt.year
    melted["MONTH"] = pd.to_datetime(melted["DATE"], format="%m/%d/%y").dt.month
    single_year_dfs.append(melted)


Processing 2019
Data frame has 117 rows
Quality columns ['quality_20190910', 'quality_20191023']

Processing 2020
Data frame has 117 rows
Quality columns ['quality_20200425', 'quality_20200527', 'quality_20200618', 'quality_20200917', 'quality_20201015', 'quality_drought_20200713', 'quality_recovery_20200722', 'quality_drought_20200810', 'quality_recovery_20200820']

Processing 2021
Data frame has 117 rows
Quality columns ['quality_20210420', 'quality_20210602', 'quality_drought_20210714', 'quality_drought_20210812', 'quality_recovery_20210927', 'quality_drought_20210729', 'quality_drought_20210812.1']

Processing 2022
Data frame has 117 rows
Quality columns ['quality_20220523', 'quality_20220609', 'quality_20220908', 'quality_20221003', 'quality_drought_20220627', 'quality_drought_20220711', 'quality_recovery_20220718', 'quality_recovery_20220726', 'quality_drought_20220809', 'quality_recovery_20220815']

Processing 2023
Data frame has 117 rows
Quality columns ['quality_20230510', 'q

In [5]:
# concatenate all tables into one
all_years_dfs = pd.concat(single_year_dfs)
all_years_dfs.head()

,PLT_ID,ROW,COL,TEST_LOC,ENTRY_NAME,ENTRY_CODE,TRAIT,DATE,QUALITY,YEAR,MONTH
0,1,1,11,"Dallas, TX",Meyer,1,Turf Quality,09/10/19,6.0,2019,9
1,2,6,3,"Dallas, TX",Meyer,1,Turf Quality,09/10/19,6.0,2019,9
2,3,9,8,"Dallas, TX",Meyer,1,Turf Quality,09/10/19,6.0,2019,9
3,4,2,4,"Dallas, TX",Emerald,2,Turf Quality,09/10/19,7.0,2019,9
4,5,7,10,"Dallas, TX",Emerald,2,Turf Quality,09/10/19,7.0,2019,9


In [6]:
print (f"Final data frame has {len(all_years_dfs)} rows")
print (f"\nFinal data frame includes observations for the following dates: \n{all_years_dfs["DATE"].unique()}")
print (f"\nFinal data frame includes observations with the following Quality Ratings: \n{all_years_dfs["QUALITY"].unique()}")

Final data frame has 4095 rows

Final data frame includes observations for the following dates: 
['09/10/19' '10/23/19' '04/25/20' '05/27/20' '06/18/20' '09/17/20'
 '10/15/20' '07/13/20' '07/22/20' '08/10/20' '08/20/20' '04/20/21'
 '06/02/21' '07/14/21' '08/12/21' '09/27/21' '07/29/21' '05/23/22'
 '06/09/22' '09/08/22' '10/03/22' '06/27/22' '07/11/22' '07/18/22'
 '07/26/22' '08/09/22' '08/15/22' '05/10/23' '06/16/23' '11/12/23'
 '07/27/23' '08/16/23' '09/08/23' '10/16/23']

Final data frame includes observations with the following Quality Ratings: 
[ 6.  7.  5.  8.  9.  4.  3.  2. nan 15.  1.]


In [7]:
all_years_dfs[all_years_dfs["QUALITY"]==15]

,PLT_ID,ROW,COL,TEST_LOC,ENTRY_NAME,ENTRY_CODE,TRAIT,DATE,QUALITY,YEAR,MONTH
97,110,6,5,"Dallas, TX",FZ 1723,37,Turf Quality,04/20/21,15.0,2021,4


In [8]:
all_years_dfs[all_years_dfs["QUALITY"].isnull()]

,PLT_ID,ROW,COL,TEST_LOC,ENTRY_NAME,ENTRY_CODE,TRAIT,DATE,QUALITY,YEAR,MONTH
48,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,04/20/21,NaN,2021,4
165,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,06/02/21,NaN,2021,6
282,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,07/14/21,NaN,2021,7
399,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,08/12/21,NaN,2021,8
516,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,09/27/21,NaN,2021,9
633,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,07/29/21,NaN,2021,7
750,58,1,8,"Dallas, TX",DALZ 1807,20,Turf Quality,08/12/21,NaN,2021,8
929,3,9,8,"Dallas, TX",Meyer,1,Turf Quality,07/26/22,NaN,2022,7
47,57,9,7,"Dallas, TX",DALZ 1806,19,Turf Quality,05/10/23,NaN,2023,5
110,3,9,8,"Dallas, TX",Meyer,1,Turf Quality,05/10/23,NaN,2023,5
